In [ ]:
#default_exp criteria

In [ ]:
#export
import warnings
import torch
import numpy as np
from torch.nn.functional import relu, softplus

from dl4to.utils import infect
from dl4to.criteria import Criterion

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Unsupervised criteria

In [ ]:
#export
class UnsupervisedCriterion(Criterion):
    """
    A parent class that inherits all unsupervised criteria for both classical and learned methods.
    """
    def __init__(
        self,
        name:str, # The name of this criterion which will be monitored in logging.
        differentiable:bool=True, # Whether the criterion is differentiable or not. Only differentiable criteria can be used as loss/objective functions.
        lower_is_better:bool=True, # Whether lower values of the criterion correspond to better scores.
        compute_only_on_design_space:bool=True # Whether the criterion should be evaluated on voxels that have a design space information of -1, i.e., the voxels can be freely optimized. This parameter does not effect all criteria.
    ):
        super().__init__(name=name, 
                         supervised=False,
                         differentiable=differentiable,
                         lower_is_better=lower_is_better,
                         compute_only_on_design_space=compute_only_on_design_space
                        )


    def _convert_to_list(self, solutions):
        if type(solutions) is tuple:
            solutions = list(solutions)
        if type(solutions) is not list:
            solutions = [solutions]
        return solutions


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        raise NotImplementedError("Must be overridden.")

In [ ]:
show_doc(UnsupervisedCriterion.__call__)

<h4 id="UnsupervisedCriterion.__call__" class="doc_header"><code>UnsupervisedCriterion.__call__</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>UnsupervisedCriterion.__call__</code>(**`solutions`**:`list`, **`gt_solutions`**:`list`=*`None`*, **`binary`**:`bool`=*`False`*)

Calculates the output of the criterion for all solutions.

||Type|Default|Details|
|---|---|---|---|
|**`solutions`**|`list`||The solutions that should be evaluated with the criterion.|
|**`gt_solutions`**|`list`|`None`|Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.|
|**`binary`**|`bool`|`False`|Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.|


In [ ]:
#export
class Compliance(UnsupervisedCriterion):
    """
    The compliance criterion which is used to determine the structural integrity of mechanical structures. 
    The criterionis computes as $F^T u$, where $F$ are the external forces and $u$ are the displacements, which are derived from the PDE for linear elasticity.
    Lower values are desired and higher values indicate worse scores.
    """
    def __init__(self, 
                 α:float=1e-9 # The weight that is used to rescale the forces F.
                ):
        self.α = α
        super().__init__(
            name=f'compliance',
            compute_only_on_design_space=False
        )


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        compliance_list = []
        for i, solution in enumerate(solutions):
            u, σ, σ_vm = solution.solve_pde(binary=binary)
            F = self.α * solution.problem.F.flatten()
            compliance_list.append(torch.dot(F, u.flatten().float()))
        return torch.stack(compliance_list)

In [ ]:
#export
class Volume(UnsupervisedCriterion):
    """
    Calculates the sum over all density values for each solution.
    """
    def __init__(self, 
                 compute_only_on_design_space:bool=False # Whether the criterion should be evaluated on voxels that have a design space information of -1, i.e., the voxels can be freely optimized. This parameter does not effect all criteria.
                ):
        super().__init__(
            name=f'volume',
            compute_only_on_design_space=compute_only_on_design_space
        )


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        θ = self.get_θ_flat(solutions, binary=binary)
        design_space_mask = self.get_design_space_mask(solutions)
        θ_design = θ * design_space_mask
        return θ_design.sum(dim=1)

In [ ]:
#export
class VolumeFraction(UnsupervisedCriterion):
    """
    Calculates the average density values for each solution. Therefore this criterion always returns values between 0 and 1.
    """
    def __init__(self, 
                 compute_only_on_design_space:bool=False # Whether the criterion should be evaluated on voxels that have a design space information of -1, i.e., the voxels can be freely optimized. This parameter does not effect all criteria.
                ):
        super().__init__(
            name=f'volume_fraction',
            compute_only_on_design_space=compute_only_on_design_space
        )
        self.volume_crit = Volume()


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        volume = self.volume_crit(solutions, gt_solutions, binary)
        design_space_mask = self.get_design_space_mask(solutions)
        return volume / design_space_mask.sum(dim=1)

In [ ]:
#export
class VolumeConstraint(UnsupervisedCriterion):
    """
    This criterion checks if the volume fraction is below a pre-defined maximum volume fraction and punishes higher volumes.
    """
    def __init__(self, 
                 max_volume_fraction:float=0.2, # The maximum volume fraction threshold given as a float between 0 and 1.
                 threshold_fct:str='softplus', # The function that determines how volume values above `max_volume_fraction` are punished. Can be either "relu" or "softplus", which is a smoothed version of ReLU.
                 compute_only_on_design_space:bool=False # Whether the criterion should be evaluated on voxels that have a design space information of -1, i.e., the voxels can be freely optimized. This parameter does not effect all criteria.
                ):
        super().__init__(
                name=f'volume_constraint',
                compute_only_on_design_space=compute_only_on_design_space
            )
        self.max_volume_fraction = max_volume_fraction
        if threshold_fct == 'softplus':
            self.threshold_fct = softplus
        elif threshold_fct == 'relu':
            self.threshold_fct = relu
        else:
            raise ValueError("`threshold_fct` must be one of ['softplus', 'relu'].")
        self.volume_fraction_crit = VolumeFraction(
            compute_only_on_design_space=compute_only_on_design_space
        )


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        volume_fraction = self.volume_fraction_crit(solutions, gt_solutions, binary)
        positives_are_too_large = volume_fraction - self.max_volume_fraction
        approximately_only_positives = self.threshold_fct(positives_are_too_large)
        return approximately_only_positives

In [ ]:
#export
class ForcesUnderpinned(UnsupervisedCriterion):
    """
    This criterion uses an infection algorithm to check if all voxels that have external forces applied to them are connected to the rest of the structure. 
    Returns 1 if the structure is connected, else it returns 0.
    """
    def __init__(self):
        super().__init__(
            name=f'forces_underpinned',
            lower_is_better=False,
            compute_only_on_design_space=False)


    def _one_channel_are_forces_underpinned(self, forces, dirichlet, density):
        assert dirichlet.dtype == density.dtype, f"{dirichlet.dtype} != {density.dtype}"
        assert dirichlet.device == density.device, f"{dirichlet.device} != {density.device}"

        underpinned = infect(dirichlet, density)
        underpinned_forces = underpinned & forces
        return torch.all(underpinned_forces == forces).item()


    def _are_forces_underpinned(self, F, Ω_dirichlet, θ):
        underpinned = True
        for i in range(3):
            if not self._one_channel_are_forces_underpinned(F[i], Ω_dirichlet[i], θ[0]):
                underpinned = False
        return underpinned


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        results = []
        if binary == False:
            warnings.warn("Automatically setting binary=True for ForcesUnderpinned Criterion.")
            binary = True
        for solution in solutions:
            θ = solution.get_θ(binary=binary) == 1
            F = solution.problem.F != 0
            Ω_dirichlet = solution.problem.Ω_dirichlet

            F = F.type(θ.dtype).to(θ.device)
            Ω_dirichlet = Ω_dirichlet.type(θ.dtype).to(θ.device)

            underpinned = self._are_forces_underpinned(F, Ω_dirichlet, θ)
            results.append(underpinned)

        return torch.tensor(results, dtype=torch.float32, device=θ.device)

In [ ]:
#export
class MaxStress(UnsupervisedCriterion):
    """
    This criterion solves the PDE for linear elasticity and returns the maximum absolute von Mises stress value for each passed solution. If `normalize=True` then the value is normalized with the yield stress of the problem.
    """
    def __init__(self, 
                 compute_only_for_not_underpinned:bool=True, # Whether not connected voxels should be included in the calculation. If True, then the maximum stress is only computed for voxels that are connected to the structure.
                 normalize:bool=True # Whether the maximal absolute von Mises stress should be normalized with the yield stress.
                ):
        super().__init__(
            name=f'max_stress',
            compute_only_on_design_space=False
        )
        self.compute_only_for_not_underpinned = compute_only_for_not_underpinned
        self.normalize = normalize
        if self.compute_only_for_not_underpinned:
            self.forces_underpinned_crit = ForcesUnderpinned()


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        if self.compute_only_for_not_underpinned:
            forces_underpinned = self.forces_underpinned_crit(solutions, gt_solutions, binary)
        else:
            forces_underpinned = len(solutions) * [1.]

        σ_vm_list = []
        σ_ys_list = []
        for i, solution in enumerate(solutions):
            if forces_underpinned[i] == 1:
                u, σ, σ_vm = solution.solve_pde(binary=binary)
                σ_vm_list.append(σ_vm.flatten())
                if self.normalize:
                    σ_ys_list.append(solution.problem.σ_ys)

        if σ_vm_list == [] and self.normalize:
            return torch.tensor([0.])

        σ_vm_ = torch.stack(σ_vm_list)
        if self.normalize:
            σ_ys_ = torch.tensor(σ_ys_list, device=σ_vm_.device)
            return σ_vm_.amax(dim=1) / σ_ys_
        return σ_vm_.amax(dim=1)

In [ ]:
#export
class StressConstraint(UnsupervisedCriterion):
    """
    This criterion solves the PDE for linear elasticity and checks if the absolute maximum von Mises stress is below the yield stress and punishes higher stress values.
    """
    def __init__(self, 
                 threshold_fct:str='softplus' # The function that determines how von Mises stress values above the yield stress are punished. Can be either "relu" or "softplus", which is a smoothed version of ReLU.
                ):
        super().__init__(
                name=f'stress_constraint',
                compute_only_on_design_space=False
            )
        if threshold_fct == 'softplus':
            self.threshold_fct = softplus
        elif threshold_fct == 'relu':
            self.threshold_fct = relu
        else:
            raise ValueError("`threshold_fct` must be one of ['softplus', 'relu'].")


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        positives_are_too_large = []
        for i, solution in enumerate(solutions):
            u,  σ, σ_vm = solution.solve_pde(binary=binary)
            positives_are_too_large.append(σ_vm - solution.problem.σ_ys)
        positives_are_too_large = torch.stack(positives_are_too_large)
        approximately_only_positives = self.threshold_fct(positives_are_too_large)
        return (approximately_only_positives ** 2).mean(dim=[1,2,3,4]) / 2

In [ ]:
#export
class Fail(UnsupervisedCriterion):
    """
    Checks for each solution if the structure fails to either underpinned force (i.e., the forces are not connected to the rest of the structure) or too high von Mises stresses.
    If the structure fails for any of these reasons, then the criterion returns a value of 1, otherwise it returns 0. 
    Averaging over the output of this criterion results in the fail percentage criterion [1].
    """
    def __init__(self,
                 ε:float=.1 # A tolerance that defines how much the absolute maximal von Mises stresses are tolerated to be slightly higher than the yield stresses. By default, this value is 0.1, i.e., a structure does still count as valid if the von Mises stresses are below 110% of the yield stress.
                ):
        super().__init__(
            name=f'fail',
            compute_only_on_design_space=False
        )
        self.ε = ε
        self.max_stress_crit = MaxStress(compute_only_for_not_underpinned=True, normalize=True)
        self.forces_underpinned_crit = ForcesUnderpinned()


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        failed_due_to_max_stress = (self.max_stress_crit(solutions, gt_solutions, binary=binary) > 1 + self.ε).float()
        failed_total = (~self.forces_underpinned_crit(solutions, gt_solutions, binary=True).bool()).float()
        failed_total.cpu()[failed_total == 0.] = failed_due_to_max_stress

        return failed_total

In [ ]:
#export
class StressEfficiency(UnsupervisedCriterion):
    """
    Returns the stress efficiency, which is calculated by dividing the mean von Mises stress through the maximum absolute von Mises stress.
    """
    def __init__(self,
                 compute_only_for_not_underpinned:bool=True # Whether not connected voxels should be included in the calculation. If True, then the stress efficiency is only computed for voxels that are connected to the structure.
                ):
        super().__init__(
            name=f'stress_efficiency',
            lower_is_better=False,
            compute_only_on_design_space=False
        )
        self.compute_only_for_not_underpinned = compute_only_for_not_underpinned
        if self.compute_only_for_not_underpinned:
            self.forces_underpinned_crit = ForcesUnderpinned()


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        if self.compute_only_for_not_underpinned:
            forces_underpinned = self.forces_underpinned_crit(solutions, gt_solutions, binary)
        else:
            forces_underpinned = len(solutions) * [1.]

        σ_vm_list = []
        for i, solution in enumerate(solutions):
            if forces_underpinned[i] == 1:
                u, σ, σ_vm = solution.solve_pde(binary=binary)
                σ_vm_list.append(σ_vm.flatten())

        if σ_vm_list == []:
            return torch.tensor([0.])

        σ_vm_ = torch.stack(σ_vm_list)
        return σ_vm_.mean(dim=1) / σ_vm_.amax(dim=1)

In [ ]:
#export
class Binariness(UnsupervisedCriterion):
    """
    This criterion is a measure for how binary a density distribution is. A value of 1 indicates that all density values are below `low` or above `high`. A value of 0 indicates that all values are between `low` and `high`.
    """
    def __init__(self, 
                 low:bool=.1, # The lower threshold below which densities are considered binary.
                 high:bool=.9, # The upper threshold above which densities are considered binary.
                 compute_only_on_design_space:bool=True # Whether the criterion should be evaluated on voxels that have a design space information of -1, i.e., the voxels can be freely optimized. This parameter does not effect all criteria.
                ):
        self.low = low
        self.high = high
        super().__init__(
            name=f'binariness',
            compute_only_on_design_space=compute_only_on_design_space)


    def __call__(self,
                 solutions:list, # The solutions that should be evaluated with the criterion.
                 gt_solutions:list=None, # Ground truth solutions that are compared element-wise with the `solutions`. Since the criterion is unsupervised this does not have an effect.
                 binary:bool=False # Whether the criterion should be evaluated on binarized densities. Does not have an effect on some criteria.
                  ):
        """
        Calculates the output of the criterion for all solutions.
        """
        solutions = self._convert_to_list(solutions)
        θ = self.get_θ_flat(solutions, binary=False)
        design_space_mask = self.get_design_space_mask(solutions)
        θ_design = θ * design_space_mask
        θ_design_filtered = torch.where(
            ((θ_design < self.low) & design_space_mask) | (θ_design > self.high), 
            torch.tensor([1.], device=θ.device),
            torch.tensor([0.], device=θ.device)
        )
        return θ_design_filtered.sum(dim=1) / design_space_mask.sum(dim=1)

# References

[1] Dittmer, Sören, et al. "SELTO: Sample-Efficient Learned Topology Optimization." arXiv preprint arXiv:2209.05098 (2022).

In [ ]:
#hide
import math
from dl4to.datasets import BasicDataset
from dl4to.problem import Problem
from dl4to.solution import Solution

In [ ]:
#hide
def get_solution(problem, enforce_θ_on_Ω_design=True):
    if problem == "ledge":
        solution = BasicDataset().ledge().trivial_solution
    elif problem == "cantilever":
        solution = BasicDataset().cantilever().trivial_solution
    elif problem == "wheel":
        solution = BasicDataset().wheel().trivial_solution

    enforce_θ_on_Ω_design = solution.enforce_θ_on_Ω_design = False
    return solution

In [ ]:
#hide
def test_it_is_unsupervised(criterion):
    assert not criterion.supervised

In [ ]:
%%time
#hide
test_it_is_unsupervised(VolumeFraction())

CPU times: user 20 µs, sys: 13 µs, total: 33 µs
Wall time: 36.5 µs


In [ ]:
%%time
#hide
test_it_is_unsupervised(StressEfficiency())

CPU times: user 18 µs, sys: 12 µs, total: 30 µs
Wall time: 33.9 µs


In [ ]:
#hide
test_it_is_unsupervised(ForcesUnderpinned())

In [ ]:
%%time
#hide

def test_that_the_output_has_the_same_shape_as_VolumeFraction():
    solution = get_solution(problem="ledge")
    solution2 = get_solution(problem="ledge")

    criterion = ForcesUnderpinned()
    result = criterion([solution])

    criterion = VolumeFraction()
    resultBCE = criterion([solution])

    assert result.shape == resultBCE.shape, f"{result.shape},{resultBCE.shape}"

    criterion = ForcesUnderpinned()
    result = criterion([solution, solution2])

    criterion = VolumeFraction()
    resultBCE = criterion([solution, solution2])

    assert result.shape == resultBCE.shape, f"{result.shape}, {resultBCE.shape}"


test_that_the_output_has_the_same_shape_as_VolumeFraction()

CPU times: user 33.8 ms, sys: 4.99 ms, total: 38.8 ms
Wall time: 57.4 ms


/tmp/ipykernel_13326/4026010179.py:41: UserWarning:

Automatically setting binary=True for ForcesUnderpinned Criterion.



In [ ]:
%%time
#hide

def test_that_ForcesUnderpinned_examples_work():
    solution0 = get_solution(problem="ledge")
    solution1 = get_solution(problem="cantilever")
    solution2 = get_solution(problem="wheel")
    solution3 = get_solution(problem="ledge")

    solution3._θ[:,10,:,:] = 0

    Ω_dirichlet = torch.zeros(3, 10, 10, 10)
    Ω_dirichlet[0, 0, 0, 0] = True
    Ω_design = -torch.ones(1, 10, 10, 10)

    F = torch.zeros(3, 10, 10, 10)
    F[0, 0, 0, 1] = 1

    problem4 = Problem(
        name="foo",
        h=np.ones(3) / 10,
        E=1,
        ν=.3,
        σ_ys=1,
        Ω_dirichlet=Ω_dirichlet,
        Ω_design=Ω_design,
        F=F
    )
    solution4 = problem4.trivial_solution
    solution4_0 = problem4.trivial_solution.clone()
    solution4_0._θ[:,0,:,:] = 0

    solution4_1 = problem4.trivial_solution.clone()
    solution4_1._θ[:,1:,:,:] = 0

    solution4_2 = problem4.trivial_solution.clone()
    solution4_2._θ[0,0,0,0] = 0

    solution4_3 = problem4.trivial_solution.clone()
    solution4_3._θ[0,0,0,1] = 0

    criterion = ForcesUnderpinned()
    result = criterion([solution0, solution1, solution2, solution4, solution4_1])
    assert result.all()

    result = criterion([solution3, solution4_0, solution4_2, solution4_3])
    assert not result.any()


test_that_ForcesUnderpinned_examples_work()#hide

CPU times: user 49.2 ms, sys: 0 ns, total: 49.2 ms
Wall time: 47.8 ms


/tmp/ipykernel_13326/4026010179.py:41: UserWarning:

Automatically setting binary=True for ForcesUnderpinned Criterion.



In [ ]:
%%time
#hide

def test_that_criterion_value_is_correct():
    solution = get_solution(problem="ledge", enforce_θ_on_Ω_design=False)

    criterion = Binariness(compute_only_on_design_space=False)

    solution.θ = .01*torch.ones_like(solution.get_θ())
    criterion_value = criterion([solution, solution])
    assert torch.allclose(criterion_value, torch.ones(2)), criterion_value

    solution.θ = .99*torch.ones_like(solution.get_θ())
    criterion_value = criterion([solution, solution])
    assert torch.allclose(criterion_value, torch.ones(2)), criterion_value

    solution.θ = .5*torch.ones_like(solution.get_θ())
    criterion_value = criterion([solution, solution])
    assert torch.allclose(criterion_value, torch.zeros(2)), criterion_value


test_that_criterion_value_is_correct()

CPU times: user 3.74 ms, sys: 0 ns, total: 3.74 ms
Wall time: 3.06 ms
